# Make Arctic Land Mask from GSHHS Coastline Data

In [1]:
from pathlib import Path
import gzip
import re

import numpy as np
import xarray as xr
import rioxarray
from pyproj import CRS, Transformer
from affine import Affine
from shapely.geometry import MultiPolygon, MultiPoint, Point

import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.cm import ScalarMappable
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from utils import latlon_from_file, true_scale_lat_to_scale_factor

from pyproj.crs import ProjectedCRS
from pyproj.crs.coordinate_operation import StereographicConversion

from ros_database.ims_snow.ims_crs import Grid

## Tasks
1. Define area of interest to include all stations in database
2. 

In [2]:
coastline = cfeature.GSHHSFeature()

In [3]:
stored_latitude, stored_longitude = latlon_from_file()

/home/apbarret/src/rain_on_snow_database/data/test_data/imslat_4km.bin.gz /home/apbarret/src/rain_on_snow_database/data/test_data/imslon_4km.bin.gz


## Define CRS and grid

In [29]:
ims_nps = StereographicConversion(latitude_natural_origin=90., longitude_natural_origin=-80.,
                                  scale_factor_natural_origin=true_scale_lat_to_scale_factor(60))
ims_crs = ProjectedCRS(ims_nps)

In [30]:
ims_crs

<Projected CRS: {"$schema": "https://proj.org/schemas/v0.2/projjso ...>
Name: undefined
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Stereographic
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
IMS4kmGrid = Grid(nrow=6144,
                  ncol=6144,
                  grid_cell_width=4000,
                  grid_cell_height=4000,
                  grid_origin_x=-12288000.0,
                  grid_origin_y=-12288000.0,
                  crs=ims_crs)

In [27]:
IMS4kmGrid.bounds()

(-12288000.0, -12288000.0, 12288000.0, 12288000.0)

In [28]:
IMS4kmGrid.transform

Affine(4000.0, 0.0, -12288000.0,
       0.0, 4000.0, -12288000.0)

In [21]:
x, y = IMS4kmGrid.xy_coords()

In [23]:
def get_latlon():
    x2d, y2d = np.meshgrid(*IMS4kmGrid.xy_coords())
    transformer = Transformer.from_crs(IMS4kmGrid.crs, 4326, always_xy=True)
    return transformer.transform(x2d, y2d)

In [24]:
%%time
lon, lat = get_latlon()

CPU times: user 10.4 s, sys: 93.2 ms, total: 10.5 s
Wall time: 10.5 s


In [ ]:
every=200
proj = ccrs.Stereographic(central_latitude=90., 
                          central_longitude=-80., 
                          true_scale_latitude=60.)
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection=proj)
ax.coastlines()

ax.scatter(lon[::every,::every], lat[::every,::every], 
           marker='+', transform=ccrs.PlateCarree())


ax.scatter(lon[0,0], lat[0,0], 
           marker='*', c='red', transform=ccrs.PlateCarree())
ax.scatter(stored_longitude[0,0], stored_latitude[0,0], 
           marker='*', c='green', transform=ccrs.PlateCarree())


In [ ]:
np.isclose(stored_latitude, lat, equal_nan=False, atol=1e-4).all()
np.isclose(stored_longitude, lon, equal_nan=False, atol=1e-4).all()

In [ ]:
print(lat[0,0], stored_latitude[0,0])
print(lat[0,-1], stored_latitude[0,-1])
print(lat[-1,-1], stored_latitude[-1,-1])
print(lat[-1,0], stored_latitude[-1,0])

print(lon[0,0], stored_longitude[0,0])
print(lon[0,-1], stored_longitude[0,-1])
print(lon[-1,-1], stored_longitude[-1,-1])
print(lon[-1,0], stored_longitude[-1,0])

In [ ]:
grid_cell_width, grid_cell_height = 4000., 4000.
#proj = to_cartopy(ims_crs)
proj = ccrs.Stereographic(central_latitude=90., 
                          central_longitude=-80., 
                          true_scale_latitude=60.)

every = 1
zoom = 1.  #0.01
extent = zoom * np.array([-1*grid_cell_width, grid_cell_width, -1*grid_cell_height, grid_cell_height])

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection=proj)
ax.set_extent(extent, proj)
ax.coastlines()
ax.gridlines()

# Add Pole
ax.scatter([0.0],[90.0], s=200, marker='o', transform=ccrs.PlateCarree(), label="Pole")

ax.scatter(stored_longitude[::every,::every], stored_latitude[::every,::every], 
           marker='P', c='k', s=100, transform=ccrs.PlateCarree())

ax.scatter(lon[::every,::every], lat[::every,::every], 
           marker='+', c='red', transform=ccrs.PlateCarree())

ax.legend(loc="lower left")

In [19]:
nrow = 6144
ncol = 6144
ngrid = nrow * ncol

data = np.fromfile("../data/test_data/imslatlon4km.6144x6144x2.double", dtype="float64")
new_lat = data[:ngrid].reshape(ncol, nrow)
new_lon = data[ngrid:].reshape(ncol, nrow)

In [25]:
print(np.isclose(new_lat, stored_latitude, equal_nan=False, atol=1e-4).all())
print(np.isclose(new_lat, lat, equal_nan=False, atol=1e-4).all())
print(new_lat[0,0], lat[0,0])
print(new_lon[0,0], lon[0,0])

False
False
-21.484801247611475 -21.488041017105115
145.0 145.0


In [ ]:
print(np.isclose(new_lon, stored_longitude, equal_nan=False, atol=1e-4).all())
print(np.isclose(new_lon, lon, equal_nan=False, atol=1e-4).all())


## Compare proj definitions

In [8]:
crs0 = CRS.from_proj4("+proj=stere +ellps=WGS84 +lat_0=90 +lon_0=-80 +lat_ts=60")
crs0

<Projected CRS: +proj=stere +ellps=WGS84 +lat_0=90 +lon_0=-80 +lat ...>
Name: unknown
Axis Info [cartesian]:
- E[south]: Easting (metre)
- N[south]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Polar Stereographic (variant B)
Datum: Unknown based on WGS 84 ellipsoid
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
crs1 = CRS.from_proj4("+proj=stere +a=6378137.0 +b=6356752.314245 +lat_0=90 +lon_0=-80 +lat_ts=60")
crs1

<Projected CRS: +proj=stere +a=6378137.0 +b=6356752.314245 +lat_0= ...>
Name: unknown
Axis Info [cartesian]:
- E[south]: Easting (metre)
- N[south]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Polar Stereographic (variant B)
Datum: Unknown based on WGS 84 ellipsoid
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
crs0.equals(crs1)

True

In [11]:
IMStoWGS84_0 = Transformer.from_crs(crs0, 4326, always_xy=True)
IMStoWGS84_1 = Transformer.from_crs(crs1, 4326, always_xy=True)

In [12]:
IMStoWGS84_0.transform(IMS4kmGrid.grid_origin_x, IMS4kmGrid.grid_origin_y)

(145.0, -21.493529831716305)

In [13]:
IMStoWGS84_1.transform(IMS4kmGrid.grid_origin_x, IMS4kmGrid.grid_origin_y)

(145.0, -21.493529831716305)

In [15]:
IMStoWGS84_0.transform(*IMS4kmGrid.xy(0.5,0.5))

(145.0, -21.484801247679794)

In [16]:
IMStoWGS84_0.transform(*IMS4kmGrid.xy(6144/2,6144/2))

(-80.0, 90.0)

In [18]:
6144/2 + 0.5

3072.5

## Get distance between new latitude and longitude and derived latitude and longitude